In [148]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import torch
from datetime import datetime, timezone
import logging

logging.basicConfig(
    level=logging.INFO,  # Set the logging level
    format='%(asctime)s - %(levelname)s - %(message)s',  # Format for the log messages
    handlers=[
        logging.StreamHandler()  # Log to the console
    ]
)

%reload_ext autoreload
%autoreload 2
from core_data_prep.core_data_prep import DataPreparer

from data.raw.retrievers.alpaca_markets_retriever import AlpacaMarketsRetriever
from config.constants import *
from data.processed.dataset_creation import DatasetCreator
from data.processed.indicators import *
from data.processed.targets import Balanced3ClassClassification
from data.processed.normalization import ZScoreOverWindowNormalizer, ZScoreNormalizer, MinMaxNormalizer
from data.processed.dataset_pytorch import DatasetPytorch
from modeling.trainer import Trainer
from modeling.evaluate import evaluate_lgb_regressor, evaluate_torch_regressor, evaluate_torch_regressor_multiasset
# from observability.mlflow_integration import log_experiment

from modeling.rl.environment import PortfolioEnvironment
from modeling.rl.state import State
from modeling.rl.agent import RlAgent
from modeling.rl.algorithms.policy_gradient import PolicyGradient
from modeling.rl.actors.actor import RlActor, FullyConnectedBackend, TransformerBackend
from modeling.rl.actors.signal_predictor_actor import SignalPredictorActor
from modeling.rl.actors.high_energy_low_friction_actor import HighEnergyLowFrictionActor
from modeling.rl.actors.xsmom_actor import XSMomActor
from modeling.rl.actors.tsmom_actor import TSMomActor
from modeling.rl.actors.blsw_actor import BLSWActor
from modeling.rl.trajectory_dataset import TrajectoryDataset
from modeling.rl.metrics import MetricsCalculator, DEFAULT_METRICS
from modeling.rl.reward import EstimatedReturnReward
from modeling.rl.loss import SumLogReturnLoss
from modeling.rl.visualization.wealth_plot import plot_cumulative_wealth
from modeling.rl.visualization.position_plot import plot_position_heatmap
from config.experiments.cur_experiment import config

torch.backends.cudnn.benchmark = config.train_config.cudnn_benchmark

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [12]:
retriever = AlpacaMarketsRetriever(download_from_gdrive=False, timeframe=config.data_config.frequency)

retrieval_result = retriever.bars_with_quotes(
    symbol_or_symbols=config.data_config.symbol_or_symbols, 
    start=config.data_config.start, 
    end=config.data_config.end)

In [149]:
data_preparer = DataPreparer(
    normalizer=config.data_config.normalizer,
    missing_values_handler=config.data_config.missing_values_handler,
    in_seq_len=config.data_config.in_seq_len,
)


In [4]:
daily_slices = data_preparer._get_daily_slices(retrieval_result, config.data_config.start, config.data_config.end)

2025-10-24 17:27:29,202 - INFO - Last timestamp counts across all slices and stocks:
2025-10-24 17:27:29,203 - INFO -   2024-09-04 16:00:00-04:00: 50 occurrences
2025-10-24 17:27:29,204 - INFO -   2024-09-05 16:00:00-04:00: 50 occurrences
2025-10-24 17:27:29,206 - INFO -   2024-09-06 16:00:00-04:00: 50 occurrences
2025-10-24 17:27:29,207 - INFO -   2024-09-09 15:59:00-04:00: 2 occurrences
2025-10-24 17:27:29,209 - INFO -   2024-09-09 16:00:00-04:00: 48 occurrences
2025-10-24 17:27:29,210 - INFO -   2024-09-10 16:00:00-04:00: 50 occurrences
2025-10-24 17:27:29,211 - INFO -   2024-09-11 16:00:00-04:00: 50 occurrences
2025-10-24 17:27:29,212 - INFO -   2024-09-12 15:59:00-04:00: 1 occurrences
2025-10-24 17:27:29,212 - INFO -   2024-09-12 16:00:00-04:00: 49 occurrences
2025-10-24 17:27:29,214 - INFO -   2024-09-13 15:59:00-04:00: 1 occurrences
2025-10-24 17:27:29,215 - INFO -   2024-09-13 16:00:00-04:00: 49 occurrences
2025-10-24 17:27:29,215 - INFO -   2024-09-16 16:00:00-04:00: 50 occurr

In [150]:
X = data_preparer.transform_data_for_inference(
    daily_slices[0], 
    n_timestamps=Constants.Data.TRADING_DAY_LENGTH_MINUTES,
    features=config.data_config.features,
)


In [151]:
X.shape

(391, 50, 60, 16)

array([[0.74336994, 0.41048747, 0.767847  , ..., 0.73275506, 0.09278772,
        0.6896251 ],
       [0.8093847 , 0.35090095, 0.7446866 , ..., 0.72328985, 0.0119793 ,
        0.71152526],
       [0.8483786 , 0.16218789, 0.7174733 , ..., 0.71500576, 0.00859069,
        0.7144389 ],
       ...,
       [0.28191018, 0.6858203 , 0.1146808 , ..., 0.        , 0.63796264,
        0.        ],
       [0.19229738, 0.724947  , 0.25536436, ..., 0.        , 0.6054872 ,
        0.        ],
       [0.11267241, 0.75197476, 0.29705158, ..., 0.        , 0.5718849 ,
        0.        ]], dtype=float32)

In [83]:
b['AAPL'][-450:]

,log_ret,hl_range,close_open,vol_delta,EMA_fast,EMA_slow,RSI2,RSI6,realvol20,VWAP_dist,loc_in_range,tod_sin,tod_cos,ema_slope,vol_slope,is_missing
174,0.757430,0.059488,0.770518,0.432594,0.982545,1.000000,0.428571,0.740000,0.812292,0.934899,1.000000,0.555117,0.0,0.844080,0.952745,0.0
175,0.272860,0.013224,0.333108,0.382738,0.928583,1.000000,0.562500,0.660714,0.844581,0.803488,0.000000,0.489297,0.0,0.724636,1.000000,0.0
176,0.363685,0.072744,0.000000,0.467346,0.850542,1.000000,0.000000,0.465116,0.844536,0.712321,0.000000,0.414710,0.0,0.593555,0.988233,0.0
177,0.212142,0.000000,0.439755,0.375879,0.696603,0.991785,0.000000,0.219512,0.947636,0.543945,0.000000,0.330098,0.0,0.395140,1.000000,0.0
178,0.212031,0.046329,0.199639,0.414041,0.504714,0.925139,0.000000,0.180000,1.000000,0.376421,0.000000,0.233988,0.0,0.179669,0.938377,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,0.528005,0.231707,0.545991,0.288455,0.759142,1.000000,0.666667,0.242117,0.923948,0.624923,1.000000,0.000000,0.0,0.353402,0.833786,0.0
620,0.535945,0.333119,0.554328,0.324884,0.822727,1.000000,1.000000,0.227945,0.936860,0.715286,0.777778,0.000000,0.0,0.448893,0.836204,0.0
621,0.407932,0.071863,0.394684,0.359964,0.862904,1.000000,1.000000,0.453258,0.902064,0.722555,0.611111,0.000000,0.0,0.497359,0.868436,0.0
622,0.000000,1.000000,0.000000,0.992193,0.712836,0.986786,0.046875,0.280346,1.000000,0.379319,0.296296,0.000000,0.0,0.254890,0.990763,0.0


In [7]:
dataset_creator = DatasetCreator(
    features=config.data_config.features,
    target=config.data_config.target,
    normalizer=config.data_config.normalizer,
    missing_values_handler=config.data_config.missing_values_handler,
    train_set_last_date=config.data_config.train_set_last_date, 
    val_set_last_date=config.data_config.val_set_last_date,
    cutoff_time=config.data_config.cutoff_time,
    in_seq_len=config.data_config.in_seq_len,
    multi_asset_prediction=config.data_config.multi_asset_prediction,
)

X_train, y_train, next_return_train, spread_train, volatility_train, \
    X_val, y_val, next_return_val, spread_val, volatility_val, \
    X_test, y_test, next_return_test, spread_test, volatility_test \
        = dataset_creator.create_dataset_numpy(retrieval_result)

X_train.shape, y_train.shape, next_return_train.shape, spread_train.shape, volatility_train.shape, \
    X_val.shape, y_val.shape, next_return_val.shape, spread_val.shape, volatility_val.shape, \
        X_test.shape, y_test.shape, next_return_test.shape, spread_test.shape, volatility_test.shape

2025-10-16 19:13:40,277 - INFO - Processing AAPL …
2025-10-16 19:13:40,594 - INFO - Imputing 496 NaN rows out of 97359 with forward fill..
2025-10-16 19:13:40,847 - INFO - Spread has 0 NaNs
2025-10-16 19:13:40,915 - INFO - Imputing 39 NaN rows with 0.5 sentinel value
2025-10-16 19:13:40,937 - INFO - Processing AMD …
2025-10-16 19:13:41,375 - INFO - Imputing 214 NaN rows out of 97359 with forward fill..
2025-10-16 19:13:41,615 - INFO - Spread has 0 NaNs
2025-10-16 19:13:41,679 - INFO - Imputing 39 NaN rows with 0.5 sentinel value
2025-10-16 19:13:41,699 - INFO - Processing BABA …
2025-10-16 19:13:42,020 - INFO - Imputing 874 NaN rows out of 97359 with forward fill..
2025-10-16 19:13:42,271 - INFO - Spread has 0 NaNs
2025-10-16 19:13:42,337 - INFO - Imputing 39 NaN rows with 0.5 sentinel value
2025-10-16 19:13:42,357 - INFO - Processing BITU …
2025-10-16 19:13:42,678 - INFO - Imputing 6493 NaN rows out of 97359 with forward fill..
2025-10-16 19:13:42,955 - INFO - Spread has 0 NaNs
2025-1

((72391, 50, 60, 15),
 (72391, 50),
 (72391, 50),
 (72391, 50),
 (72391, 50),
 (7350, 50, 60, 15),
 (7350, 50),
 (7350, 50),
 (7350, 50),
 (7350, 50),
 (7350, 50, 60, 15),
 (7350, 50),
 (7350, 50),
 (7350, 50),
 (7350, 50))

In [8]:
y_train.mean(), y_val.mean(), y_test.mean()

(0.4998529, 0.4990973, 0.5015)

In [ ]:
# 	    next_return, spread, volatility
# train (0.00062720885, 0.00025727754, 0.000789116)
# val   (0.0011397429, 0.0002572774, 0.0014156421)
# test  (0.0005497588, 0.0002572774, 0.00068560627)

In [ ]:
np.abs(next_return_train).mean(), spread_train.mean(), volatility_train.mean()

In [ ]:
np.abs(next_return_val).mean(), spread_val.mean(), volatility_val.mean()

In [ ]:
np.abs(next_return_test).mean(), spread_test.mean(), volatility_test.mean()

In [ ]:
train_loader = DatasetPytorch(X_train, y_train, learning_task='regression').as_dataloader(
    batch_size=config.train_config.batch_size,
    shuffle=config.train_config.shuffle,
    num_workers=config.train_config.num_workers,
    prefetch_factor=config.train_config.prefetch_factor,
    pin_memory=config.train_config.pin_memory,
    persistent_workers=config.train_config.persistent_workers,
    drop_last=config.train_config.drop_last
)
val_loader = DatasetPytorch(X_val, y_val, learning_task='regression').as_dataloader(
    batch_size=config.train_config.batch_size,
    shuffle=config.train_config.shuffle,
    num_workers=config.train_config.num_workers,
    prefetch_factor=config.train_config.prefetch_factor,
    pin_memory=config.train_config.pin_memory,
    persistent_workers=config.train_config.persistent_workers,
    drop_last=config.train_config.drop_last
)

In [7]:
model = config.model_config.model
model

TemporalSpatial(
  (asset_embed): Embedding(50, 16)
  (asset_proj): Linear(in_features=16, out_features=256, bias=False)
  (lstm): LSTM(15, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (spatial_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [8]:
config

ExperimentConfig(data_config=DataConfig(symbol_or_symbols=['AAPL', 'AMD', 'BABA', 'BITU', 'C', 'CSCO', 'DAL', 'DIA', 'GLD', 'GOOG', 'IJR', 'MARA', 'MRVL', 'MU', 'NEE', 'NKE', 'NVDA', 'ON', 'PLTR', 'PYPL', 'QLD', 'QQQ', 'QQQM', 'RKLB', 'RSP', 'SMCI', 'SMH', 'SOXL', 'SOXX', 'SPXL', 'SPY', 'TMF', 'TNA', 'TQQQ', 'TSLA', 'UBER', 'UDOW', 'UPRO', 'VOO', 'WFC', 'XBI', 'XLC', 'XLE', 'XLI', 'XLK', 'XLU', 'XLV', 'XLY', 'XOM', 'XRT'], start=datetime.datetime(2024, 6, 1, 0, 0), end=datetime.datetime(2025, 6, 1, 0, 0), features={'log_ret': <function <lambda> at 0x7f96b70f6ca0>, 'hl_range': <function <lambda> at 0x7f96b70f6de0>, 'close_open': <function <lambda> at 0x7f96b6f08a40>, 'vol_delta': <function <lambda> at 0x7f96b6f08ae0>, 'EMA_fast': <data.processed.indicators.EMA object at 0x7f96b70bda90>, 'EMA_slow': <data.processed.indicators.EMA object at 0x7f96b709cb90>, 'RSI2': <data.processed.indicators.RSI object at 0x7f96b704ff50>, 'RSI6': <data.processed.indicators.RSI object at 0x7f96c720af90>, '

In [ ]:
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    loss_fn=config.train_config.loss_fn,
    optimizer=config.train_config.optimizer,
    scheduler=config.train_config.scheduler,
    num_epochs=config.train_config.num_epochs,
    early_stopping_patience=config.train_config.early_stopping_patience,
    device=config.train_config.device,
    metrics=config.train_config.metrics,
    save_path=config.train_config.save_path
)

In [ ]:
# 0.1154
# 0.3397

In [11]:
model, history = trainer.train()

2025-08-11 16:14:37,973 - INFO - Epoch 1/20
2025-08-11 16:15:24,477 - INFO - Train Loss: 0.1358        
2025-08-11 16:15:24,477 - INFO - Train Rmse: 0.3670
2025-08-11 16:15:24,477 - INFO - Val   Loss: 0.1219
2025-08-11 16:15:24,477 - INFO - Val   Rmse: 0.3490
2025-08-11 16:15:24,477 - INFO - 
2025-08-11 16:15:24,479 - INFO - Epoch 2/20
2025-08-11 16:16:05,813 - INFO - Train Loss: 0.1253        
2025-08-11 16:16:05,814 - INFO - Train Rmse: 0.3539
2025-08-11 16:16:05,814 - INFO - Val   Loss: 0.1171
2025-08-11 16:16:05,814 - INFO - Val   Rmse: 0.3421
2025-08-11 16:16:05,815 - INFO - 
2025-08-11 16:16:05,816 - INFO - Epoch 3/20
2025-08-11 16:16:47,121 - INFO - Train Loss: 0.1240        
2025-08-11 16:16:47,121 - INFO - Train Rmse: 0.3521
2025-08-11 16:16:47,122 - INFO - Val   Loss: 0.1160
2025-08-11 16:16:47,122 - INFO - Val   Rmse: 0.3405
2025-08-11 16:16:47,122 - INFO - 
2025-08-11 16:16:47,123 - INFO - Epoch 4/20
2025-08-11 16:17:28,418 - INFO - Train Loss: 0.1235        
2025-08-11 16:

In [ ]:
train_trajectory_loader = TrajectoryDataset(X_train, next_return_train, spread_train, volatility_train, trajectory_length=16).as_dataloader(
    batch_size=8, 
    shuffle=True,
    num_workers=8,
    prefetch_factor=4,
    pin_memory=True,
    persistent_workers=True,
    drop_last=True,
)
val_trajectory_loader = TrajectoryDataset(X_val, next_return_val, spread_val, volatility_val, trajectory_length=16).as_dataloader(
    batch_size=8, 
    shuffle=False,
    num_workers=8,
    prefetch_factor=4,
    pin_memory=True,
    persistent_workers=True,
    drop_last=True,
)
test_trajectory_loader = TrajectoryDataset(X_test, next_return_test, spread_test, volatility_test, trajectory_length=16).as_dataloader(
    batch_size=8, 
    shuffle=False,
    num_workers=8,
    prefetch_factor=4,
    pin_memory=True,
    persistent_workers=True,
    drop_last=True,
)

In [ ]:
env = PortfolioEnvironment(
    reward_function=EstimatedReturnReward(fee=0.0, spread_multiplier=0.99),
)

backend = FullyConnectedBackend(
    n_assets=len(config.data_config.symbol_or_symbols),
    hidden_dim=128,
    num_layers=2, 
    dropout=0.1,
    use_layer_norm=False,
)

actor = RlActor(
    model, 
    backend,
    n_assets=len(config.data_config.symbol_or_symbols),
    train_signal_predictor=False, 
    exploration_eps=0.0
).to(device)

signal_predictor_actor = SignalPredictorActor(
    model, 
    trade_asset_count=1,
    train_signal_predictor=False
).to(device)

rl_agent = RlAgent(
    actor, 
    env,
    single_action_per_trajectory=False
)

metrics_calculator = MetricsCalculator(
    metrics=DEFAULT_METRICS
)

policy_gradient = PolicyGradient(
    rl_agent, 
    train_trajectory_loader, 
    val_trajectory_loader, 
    metrics_calculator=metrics_calculator,
    optimizer=torch.optim.AdamW(
        [p for p in actor.parameters() if p.requires_grad], 
        lr=1e-3,
        weight_decay=1e-5,
        amsgrad=True),
    scheduler=None,
    loss_fn=SumLogReturnLoss(use_baseline=False),
    num_epochs=10,
    device=device
)

In [ ]:
print('Val set evaluation')
epoch_loss, realized_returns_signal_predictor, actions_signal_predictor = policy_gradient.evaluate(signal_predictor_actor)

2025-08-11 16:32:01,912 - INFO - [PolicyGradient] [VAL] Epoch 0/10 — CumulativeReturn: 0.4364, MeanReturnPercentage: 0.0054


[PolicyGradient] [VAL] Epoch 0/10 — Loss: -0.0453


In [ ]:
print('Test set evaluation')
epoch_loss, realized_returns_signal_predictor, actions_signal_predictor = policy_gradient.evaluate(signal_predictor_actor, test_trajectory_loader)

In [17]:
plot_cumulative_wealth(
    returns_dict={
        'Signal Predictor': realized_returns_signal_predictor,
    }, 
    start_time=config.data_config.train_set_last_date, 
    end_time=config.data_config.end
)

TypeError: can't subtract offset-naive and offset-aware datetimes

In [24]:
import copy  # Local import to avoid polluting global namespace unnecessarily
state_dict = (
    model.module.state_dict()
        if isinstance(model, torch.nn.DataParallel)
    else model.state_dict()
)

# Keep a local copy of the best weights so we can return the best model
# after training finishes, without needing to reload from disk.
best_model_state = copy.deepcopy(state_dict)

# Persist to disk if a save_path was provided
torch.save(state_dict, "best_model.pth")

In [20]:
log_experiment(
    config=config, 
    model=model, 
    history=history,
    input_data_sample=next(iter(train_loader))[0].to(trainer.device))

2025/06/26 15:35:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'LSTM Default' already exists. Creating a new version of this model...
2025/06/26 15:35:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LSTM Default, version 10


🏃 View run gentle-loon-699 at: http://127.0.0.1:8080/#/experiments/439216085822475480/runs/54deb1104660468d9ffb4e7e278e9cfb
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/439216085822475480


Created version '10' of model 'LSTM Default'.


In [ ]:
evaluate_lgb_regressor(X_train, y_train, X_val, y_val, next_return_val)

c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000873 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9435
[LightGBM] [Info] Number of data points in the train set: 7371, number of used features: 37
[LightGBM] [Info] Start training from score 0.497863
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
